# 모델 성능 평가를 위한 데이터 셋 분리
## 데이터셋(Dataset)
- Train 데이터셋 (훈련 / 학습 데이터셋)
    - 모델을 학습시킬 때 사용할 데이터셋
- Validation 데이터셋 (검증 데이터셋)
    - 모델의 성능 중간 검증을 위한 데이터셋
- Test 데이터셋 (평가 데이터셋)
    - 모델의 성능을 최종적으로 측정하기 위한 데이터셋
    - `Test 데이터셋은 마지막에 모델의 성능을 측정하는 용도로 한번만 사용`

## Hold Out - Data분리 방식

![image](https://blog.kakaocdn.net/dn/R5F4R/btqHb0lGapZ/Cupp4u4JJLbKXKKhfgfhM1/img.png)
- 데이터셋을 Train set, Validation set, Test set으로 나눔
- sklearn.model_selecton.train_test_split() 함수 사용
    - 하나의 데이터셋을 2분할 하는 함수

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [2]:
X, y = load_iris(return_X_y = True)
X.shape, y.shape

((150, 4), (150,))

### Train / Test set 분리

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.2,
                                                    stratify = y,
                                                    random_state = 0
                                                    )
X_train.shape, X_test.shape

((120, 4), (30, 4))

### Train / Validation / Test set 분리

In [4]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = 0.2, stratify = y_train, random_state = 0)
print(X_train.shape, X_val.shape, X_test.shape)

(96, 4) (24, 4) (30, 4)


### 모델 생성, 평가
- max_depth = 정수
    - 하이퍼파라미터 : 모델의 성능에 영향을 주는 파라미터 값으로 개발자가 설정하는 값
    - 파라미터 : 머신러닝 모델의 파라미터 -> 학습을 통해서 찾는 모델의 가중치 값

In [5]:
# 모델 생성
tree = DecisionTreeClassifier(max_depth = 5, random_state = 0)

# 학습
tree.fit(X_train, y_train)

# 검증
pred_train = tree.predict(X_train)
pred_val = tree.predict(X_val)

# 정확도 계산
train_acc = accuracy_score(y_train, pred_train)
val_acc = accuracy_score(y_val, pred_val)

In [6]:
print("max_depth:", 5)
print("train 정확도:", train_acc)
print("val 정확도:", val_acc)

max_depth: 5
train 정확도: 1.0
val 정확도: 1.0


### Testset으로 최종검증

In [7]:
pred_test = tree.predict(X_test)
test_acc = accuracy_score(y_test, pred_test)
print("Testset 정확도: ", test_acc)

Testset 정확도:  0.9666666666666667


### Hold out 방식의 단점
- train / validation / test set이 어떻게 나눠 지냐에 따라 결과가 다름
    - 데이터가 적을 경우 문제가 발생할 수 있다
        - 이상치에 대한 영향을 많이 받음
        - 다양한 패턴을 찾을 수 없어 새로운 데이터에 대한 예측 능력 떨어짐
- Hold out방식은 데이터의 양이 많을 때 사용

## K-겹 교차검증 (K-Fold Cross Validation) - Data 분리 방식
1. 데이터셋을 설정한 K개로 나눔
2. K개 중 하나를 검증세트로 나머지를 훈련세트로 하여 모델을 학습하고 평가
3. K개 모두 한번씩 검증 세트가 되도록 K번 반복해 모델을 학습, 평가지표들을 평균내서 모델의 성능을 평가

![image](https://blog.kakaocdn.net/dn/br92jB/btrnwoqmQVy/qAr76wPpnYkoveJaZAXXHk/img.png)

- 데이터양이 충분치 않을 때 사용
- 보통 Fold를 나눌 때 2.5 : 7.5 또는 2 : 8 비율이 되게 하기 위해 4개 또는 5개 fold로 나눈다
- 종류
    - KFold
        - 회귀문제의 Dataset을 분리할 때 사용 (순서대로 동일한 개수로 나눔)
    - StratifiedKFold
        - 분류문제의 Dataset을 분리할 때 사용 (동일한 비율로 나눔)

> ### Boston Housing DataSet
> 보스톤의 지역별 집값 데이터셋
> 
>  - CRIM	: 지역별 범죄 발생률
>  - ZN	: 25,000 평방피트를 초과하는 거주지역의 비율
>  - INDUS: 비상업지역 토지의 비율
>  - CHAS	: 찰스강에 대한 더미변수(강의 경계에 위치한 경우는 1, 아니면 0)
>  - NOX	: 일산화질소 농도
>  - RM	: 주택 1가구당 평균 방의 개수
>  - AGE	: 1940년 이전에 건축된 소유주택의 비율
>  - DIS	: 5개의 보스턴 고용센터까지의 접근성 지수
>  - RAD	: 고속도로까지의 접근성 지수
>  - TAX	: 10,000 달러 당 재산세율
>  - PTRATIO : 지역별 교사 한명당 학생 비율
>  - B	: 지역의 흑인 거주 비율
>  - LSTAT: 하위계층의 비율(%)
>  
>  - MEDV	: Target.  지역의 주택가격 중앙값 (단위: $1,000)
>

In [9]:
from sklearn.datasets import load_boston
import numpy as np
import pandas as pd

boston = load_boston()
data = boston.data
target = boston.target

c:\Users\Administrator\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing d

In [10]:
data.shape, target.shape

((506, 13), (506,))

In [11]:
target[:20], np.unique(target, return_counts = True)

(array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
        18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2]),
 (array([ 5. ,  5.6,  6.3,  7. ,  7.2,  7.4,  7.5,  8.1,  8.3,  8.4,  8.5,
          8.7,  8.8,  9.5,  9.6,  9.7, 10.2, 10.4, 10.5, 10.8, 10.9, 11. ,
         11.3, 11.5, 11.7, 11.8, 11.9, 12. , 12.1, 12.3, 12.5, 12.6, 12.7,
         12.8, 13. , 13.1, 13.2, 13.3, 13.4, 13.5, 13.6, 13.8, 13.9, 14. ,
         14.1, 14.2, 14.3, 14.4, 14.5, 14.6, 14.8, 14.9, 15. , 15.1, 15.2,
         15.3, 15.4, 15.6, 15.7, 16. , 16.1, 16.2, 16.3, 16.4, 16.5, 16.6,
         16.7, 16.8, 17. , 17.1, 17.2, 17.3, 17.4, 17.5, 17.6, 17.7, 17.8,
         17.9, 18. , 18.1, 18.2, 18.3, 18.4, 18.5, 18.6, 18.7, 18.8, 18.9,
         19. , 19.1, 19.2, 19.3, 19.4, 19.5, 19.6, 19.7, 19.8, 19.9, 20. ,
         20.1, 20.2, 20.3, 20.4, 20.5, 20.6, 20.7, 20.8, 20.9, 21. , 21.1,
         21.2, 21.4, 21.5, 21.6, 21.7, 21.8, 21.9, 22. , 22.1, 22.2, 22.3,
         22.4, 22.5, 22.6, 22.7, 22.8

In [12]:
np.mean(target), np.median(target)

(22.532806324110677, 21.2)

## KFold
- 지정한 개수(K)만큼 분할
- Raw dataset의 순서를 유지하면서 지정한 개수로 분할
- 회귀 문제일때 사옹
- KFold(n_splits = K)
    - 몇개의 Fold로 나눌지 지정
- KFold객체.split(데이터셋)
    - 데이터셋을 지정한 K개 나눴을때 train/test set에 포함될 데이터의 index들을 반환하는 generator 생성

> - Generator란
>     - 연속된 값을 제공(생성)하는 객체. 연속된 값을 한번에 메모리에 저장하지 않고 필요시마다 순서대로 하나씩 제공한다.
>     - 함수형식으로 구현하며 return 대신 yield를 사용한다.

### KFold 예제
#### Boston housing dataset을 KFold를 이용해 나누기

In [13]:
from sklearn.model_selection import KFold

kfold = KFold(n_splits = 5) # K:4 => 7.5 : 2.5, K:5 => 8:2
gen = kfold.split(data)
print(type(gen))

<class 'generator'>


In [14]:
next(gen)

(array([102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
        115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127,
        128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140,
        141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153,
        154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166,
        167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179,
        180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192,
        193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205,
        206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218,
        219, 220, 221, 222, 223, 224, 225, 226, 227, 228, 229, 230, 231,
        232, 233, 234, 235, 236, 237, 238, 239, 240, 241, 242, 243, 244,
        245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257,
        258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270,
        271, 272, 273, 274, 275, 276, 277, 278, 279

In [20]:
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

In [21]:
# KFold를 이용해 교차 검증
val_results = []

kfold = KFold(n_splits=5)
gen = kfold.split(data)

for train_idx, test_idx in gen:
    X_train, y_train = data[train_idx], target[train_idx]
    X_test, y_test = data[test_idx], target[test_idx]
    
    # 모델 생성, 학습
    tree = DecisionTreeRegressor(random_state = 0)
    tree.fit(X_train, y_train)
    
    # 평가
    pred_test = tree.predict(X_test)
    test_mse = mean_squared_error(y_test, pred_test)
    
    val_results.append(test_mse)

In [22]:
print("fold별 평가결과: ", val_results)
print("최종결과: ", np.mean(val_results), np.sqrt(np.mean(val_results)))

fold별 평가결과:  [11.887843137254906, 34.88990099009901, 28.17247524752476, 54.44178217821782, 52.59029702970297]
최종결과:  36.39645971655989 6.032947846331832


In [23]:
# iris 데이터셋을 KFold 이용해서 cross validation
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
X, y = load_iris(return_X_y = True)

kfold_iris = KFold(n_splits = 3)
for train_idx, test_idx in kfold_iris.split(X):
    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]
    # 모델 생성 및 학습
    tree = DecisionTreeClassifier(random_state = 0)
    tree.fit(X_train, y_train)
    # 검증
    pred_test = tree.predict(X_test)
    print(accuracy_score(y_test, pred_test))

0.0
0.0
0.0


## StratifiedKFold
- 나뉜 fold들에 label들이 같은(또는 거의 같은) 비율로 구성 되도록 나눈다
- 각각의 클래스 별로 각각 순서대로 나눈다
- 분류 문제일 때 사용
- StrarifiedKFold(n_splits = K)
    - 몇개의 Fold로 나눌지 지정
- StratifiedKFold객체.split(X, y)
    - 데이터셋을 지정한 K개 나눴을때 train/test set에 포함될 데이터의 index들을 반환하는 generator 생성
    - input(X)와 output(y) dataset을 전달

In [24]:
from sklearn.datasets import load_iris
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [25]:
X, y = load_iris(return_X_y = True)

s_kfold = StratifiedKFold(n_splits = 3)
gen = s_kfold.split(X, y)
val_results = []

for train_idx, test_idx in gen:
    X_train, y_train = X[train_idx], y[train_idx]
    X_test, y_test = X[test_idx], y[test_idx]
    
    print(np.unique(y_train, return_counts = True))
    
    # 모델 생성
    tree = DecisionTreeClassifier(random_state = 0)
    
    # 학습
    tree.fit(X_train, y_train)
    
    # 검증
    ## 추론
    pred_test = tree.predict(X_test)
    acc = accuracy_score(y_test, pred_test)
    val_results.append(acc)

(array([0, 1, 2]), array([33, 33, 34], dtype=int64))
(array([0, 1, 2]), array([33, 34, 33], dtype=int64))
(array([0, 1, 2]), array([34, 33, 33], dtype=int64))


In [26]:
val_results, np.mean(val_results)

([0.98, 0.94, 0.98], 0.9666666666666667)

In [30]:
def cross_validation_function(X, y, model, n_splits = 5):
    s_kfold = StratifiedKFold(n_splits = n_splits)
    gen = s_kfold.split(X, y)
    
    val_results = []
    
    for train_idx, test_idx in gen:
        X_train, y_train = X[train_idx], y[train_idx]
        X_test, y_test = X[test_idx], y[test_idx]
        
        # 학습 
        model.fit(X_train, y_train)
        
        # 검증
        ## 추론
        pred_test = model.predict(X_test)
        acc = accuracy_score(y_test, pred_test)
        val_results.append(acc)
        
    return val_results

In [31]:
X, y = load_iris(return_X_y = True)
model = DecisionTreeClassifier(random_state = 0)
result = cross_validation_function(X, y, model, n_splits=4)
print(result)
print(np.mean(result))

[0.9736842105263158, 0.9473684210526315, 0.9459459459459459, 1.0]
0.9667496443812233


## cross_val_score()
- 데이터셋을 K개로 나누고 K번 반복하면서 평가하는 작업을 처리해 주는 함수
- 주요매개변수
    - estimator : 모델 객체
    - X : feature
    - y : label
    - scoring : 평가지표
    - cv : 나눌 개수 (K)
        - 정수 : 개수
        - KFold 타입 객체
- 반환값 : array - 각 반복마다의 평가점수

In [32]:
# Boston dataset(회귀)
# neg_mean_squared_error : 계산한 오차평균에 음수를 붙여서 반환
from sklearn.model_selection import cross_val_score

tree_reg = DecisionTreeRegressor(random_state = 0)
result1 = cross_val_score(estimator = tree_reg,
                          X = data, 
                          y = target, 
                          scoring = 'neg_mean_squared_error',
                          cv = 4
                          )

In [33]:
print(-result1)
print(np.mean(-result1))

[13.70937008 66.44086614 52.79198413 41.87055556]
43.70319397575304


In [34]:
# iris datset(분류)
tree_cls = DecisionTreeClassifier(random_state = 0)
result2 = cross_val_score(estimator = tree_cls,
                          X = X,
                          y = y,
                          scoring = 'accuracy',
                          cv = 4
                          )

In [35]:
print(result2)
print(np.mean(result2))

[0.97368421 0.94736842 0.94594595 1.        ]
0.9667496443812233
